# EDA

- Exploring Google and Trustpilot datasets

In [ ]:
import pandas as pd

In [ ]:
def extract(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    print(f"Loaded {file_path}: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

In [ ]:
google_path = "../data/raw/google_reviews.csv"
df_google = extract(google_path)

In [ ]:
df_google.head()

In [ ]:
trustpilot_path = "../data/raw/trustpilot_reviews.csv"
df_trustpilot = extract(trustpilot_path)

In [ ]:
df_trustpilot.head()

In [ ]:
print(
    f" There are {df_google['Comment'].isnull().sum()} missing customer reviews in the Google data set"
)
print(
    f" There are {df_trustpilot['Review Content'].isnull().sum()} missing customer reviews in the Trustpilot data set"
)

In [ ]:
# Find the number of unique locations in the Google data set.

google_locations = set(df_google["Club's Name"])
print(f"There are {len(google_locations)} unique locations in the Google data set")

In [ ]:
# Find the number of unique locations in the Trustpilot data set.

trustpilot_locations = set(df_trustpilot["Location Name"])
print(
    f"There are {len(trustpilot_locations)} unique locations in the Trustpilot data set"
)

In [ ]:
# Find the number of common locations between the Google data set and the Trustpilot data set.

common_locations = set(google_locations).intersection(set(trustpilot_locations))
print(
    f"There are {len(common_locations)} common locations between the Google data set and the Trustpilot data set"
)

In [ ]:
df_google.drop_duplicates(inplace=True)
df_trustpilot.drop_duplicates(inplace=True)

df_google.dropna(subset=["Comment"], inplace=True)
df_trustpilot.dropna(subset=["Review Content"], inplace=True)

df_google.reset_index(drop=True, inplace=True)
df_trustpilot.reset_index(drop=True, inplace=True)

## Visualisations

In [ ]:
%pip install matplotlib wordcloud nltk
print("Installation completed")

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download("all", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("punkt_tab", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)
nltk.download("averaged_perceptron_tagger_eng", quiet=True)

In [ ]:
stop_words = set(stopwords.words("english"))
stop_word_additions = ["gym", "club"]
stop_words.update(stop_word_additions)

lemmatizer = WordNetLemmatizer()
tokenizer = word_tokenize

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
def tokenize_text(text):
    tokens = tokenizer(text)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [ ]:
def lemmatize_tokens(tokens):
    # convert POS tag to WordNet format
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {
            "J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV,
        }
        return tag_dict.get(tag, wordnet.NOUN)

    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]

    return lemmas

In [ ]:
def text_processing(df, column_name):
    df["clean_review"] = df[column_name].apply(clean_text)
    df["clean_review"] = df["clean_review"].apply(tokenize_text)
    df["clean_review"] = df["clean_review"].apply(lemmatize_tokens)
    return df

In [ ]:
df_google_clean = text_processing(df_google, "Comment")
df_google_clean.head()

In [ ]:
df_google_clean["Comment"][0]

In [ ]:
df_trustpilot_clean = text_processing(df_trustpilot, "Review Content")
df_trustpilot_clean.head()

In [ ]:
def flatten_tokens(token_lists):
    return [token for sublist in token_lists for token in sublist]

In [ ]:
def generate_wordcloud(df, column_name, title="Word Cloud"):
    all_tokens = flatten_tokens(df[column_name].dropna())
    text = " ".join(all_tokens)

    wordcloud = WordCloud(
        width=800, height=400, background_color="white", min_font_size=10
    ).generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title, fontsize=16)
    plt.show()

In [ ]:
generate_wordcloud(df_google_clean, "clean_review", title="Google Reviews WordCloud")

In [ ]:
generate_wordcloud(
    df_trustpilot_clean, "clean_review", title="Trustpilot Reviews WordCloud"
)

In [ ]:
def plot_word_frequency(df, column_name, top_n=20):
    all_tokens = flatten_tokens(df[column_name].dropna())

    fdist = FreqDist(all_tokens)
    most_common = fdist.most_common(top_n)
    words, counts = zip(*most_common)

    plt.figure(figsize=(12, 6))
    plt.bar(words, counts)
    plt.xticks(rotation=45)
    plt.title(f"Top {top_n} Words Frequency")
    plt.show()

In [ ]:
plot_word_frequency(df_google_clean, "clean_review")

In [ ]:
plot_word_frequency(df_trustpilot_clean, "clean_review")

# Exploring negative reviews

In [ ]:
google_rename_dict = {
    "Social Media Source": "source",
    "Club's Name": "location",
    "Creation Date": "date_created",
    "Comment": "review",
    "Overall Score": "score",
}

In [ ]:
trustpilot_rename_dict = {
    "Source Of Review": "source",
    "Location Name": "location",
    "Review Created (UTC)": "date_created",
    "Review Content": "review",
    "Review Stars": "score",
}

In [ ]:
def transform(df: pd.DataFrame, rename_dict):

    # Rename columns
    df = df.rename(columns=rename_dict)

    # Select only the renamed columns
    columns_to_keep = list(rename_dict.values())
    df = df[columns_to_keep].copy()

    # Filter on rating
    max_rating = 3
    df = df[df["score"] <= max_rating]

    return df

In [ ]:
transformed_google = transform(df_google, google_rename_dict)

In [ ]:
transformed_google.head()

In [ ]:
transformed_trustpilot = transform(df_trustpilot, trustpilot_rename_dict)

In [ ]:
transformed_trustpilot.head()

In [ ]:
print("Google:")
print(transformed_google.dtypes)
print("\nTrustpilot:")
print(transformed_trustpilot.dtypes)

In [ ]:
combined_df = pd.concat([transformed_google, transformed_trustpilot], ignore_index=True)
combined_df.head()

In [ ]:
dtype_dict = {
    "source": "string",
    "location": "string",
    "review": "string",
    "score": "int64",
}

In [ ]:
for col, dtype in dtype_dict.items():
    combined_df[col] = combined_df[col].astype(dtype)

combined_df["date_created"] = pd.to_datetime(
    combined_df["date_created"], errors="coerce"
)

In [ ]:
combined_df_cleaned = text_processing(combined_df, "review")
combined_df_cleaned.head()

In [ ]:
generate_wordcloud(
    combined_df_cleaned, "clean_review", title="Combined Reviews WordCloud"
)

In [ ]:
plot_word_frequency(combined_df_cleaned, "clean_review")

- The combined negative dataset could provide the business with key insights into how to improve their services for their customers through topic and sentiment analysis

- EDA of the real data revealed that London had the most negative reviews. A potential trial rollout of improvements at most negatively reviewed gyms would have the biggest impact. 